In [1]:
Today = '2024-03-31'

print_Deals = True

In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [4]:
query = f"""select * from RISKACCESS."sapDeals" where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') """
data_ex_deals_export = export_from_RISKCUSTOM(query)

query = f"""select * from RISKACCESS."sapIcDeals" where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') """
data_in_deals_export = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [28]:
work_data_in_deals
data_in_deals_export.query("dealStatusName != 'Cancelled' & invoiced == 'Y' & deliveryPeriodTo > @Today").reset_index(drop=True)


,##batch_id,##deleted,##inserted,##origin_name,pricingKey,conditionItemNumber,conditionType,reportDate,conditionGroup,id,...,ecvnar,invoiced,cashed,billingTypeId,billingType,producerId,producerName,dealDate,deliveryLotToleranceOption,priceFixingDate


In [24]:
work_data_list = [data_ex_deals_export, data_in_deals_export]

for work_data_index in range(len(work_data_list)):
    data_work = work_data_list[work_data_index]
    data_work = data_work.query("dealStatusName != 'Cancelled' & invoiced == 'Y' & deliveryPeriodTo > @Today").reset_index(drop=True)
    data_work[['sellerName', 'buyerName']] = data_work[['sellerName', 'buyerName']].fillna('')
    data_work['Counterparty'] = data_work['sellerName'] + data_work['buyerName']
    data_work.loc[data_work['actualQuantity'].isna(), 'actualQuantity'] = data_work.loc[data_work['actualQuantity'].isna(), 'BLTonnage']
    if work_data_index == 0:
        price_col = 'conditionPrice'
    else:
        price_col = 'price'
    data_work['Sum_money'] = data_work['actualQuantity'] * data_work[price_col]
    data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'actualQuantity'] = data_work.actualQuantity.abs()
    data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'Sum_money'] = -1 * data_work.Sum_money.abs()
    data_work.loc[data_work.dealTypeName == 'Sales Deal', 'actualQuantity'] = -1 * data_work.actualQuantity.abs()
    data_work.loc[data_work.dealTypeName == 'Sales Deal', 'Sum_money'] = data_work.Sum_money.abs()
    data_work['CompCode'] = merge_Mapping(data_work, col='companyName')
    data_work['Business_segment'] = merge_SalesUnits(data_work, col='CompCode', merge_col='ocpSegment')
    data_work['Cpty_Code'] = merge_Mapping(data_work, col='Counterparty')
    data_work['Cpty_segment'] = merge_SalesUnits(data_work, col='Cpty_Code', merge_col='ocpSegment').fillna('External')
    data_work = Period(data_work, day_for_count=Today, col_with_date='deliveryPeriodTo')
    data_work['holding'] = merge_SalesUnits(data_work, 'CompCode', merge_col='holding')

    data_for_print = data_work[['productTypeName', 'companyName', 'Business_segment', 'Counterparty',  'Cpty_segment', 'currency', 'actualQuantity', 'Sum_money', 'deliveryPeriodTo', 'Days', 'Period']]

    if work_data_index == 0:
        work_data_ex_deals = data_for_print
    else:
        work_data_in_deals = data_for_print

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\

In [25]:
Sheet_in_output_file = 'Deals'

if print_Deals == True:
    for group in ['EUROCHEM', 'SUEK']:
        work_data_ex_deals_group = work_data_ex_deals[work_data_ex_deals.holding == group]
        work_data_in_deals_group = work_data_in_deals[work_data_in_deals.holding == group]
        Output_file_ex = "_".join([str(date.today()), group, 'EX_Deals.xlsx'])
        Output_file_in = "_".join([str(date.today()), group, 'IN_Deals.xlsx'])
        work_data_ex_deals_group.to_excel(Output_file_ex, Sheet_in_output_file, index=False)
        work_data_in_deals_group.to_excel(Output_file_in, Sheet_in_output_file, index=False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9880\3230448435.py:9: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  work_data_ex_deals_group.to_excel(Output_file_ex, Sheet_in_output_file, index=False)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9880\3230448435.py:10: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  work_data_in_deals_group.to_excel(Output_file_in, Sheet_in_output_file, index=False)
